In [2]:
!nvidia-smi

Sun May  1 08:07:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Directory settings

In [3]:
# ====================================================
# Directory settings
# ====================================================

import sys
print(sys.version)
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount("/content/drive")
    base = "/content/drive/MyDrive/colab_notebooks/kaggle/nbme-score-clinical-patient-notes/notebooks"
    %cd {base}


import os
if 'kaggle_web_client' in sys.modules:
    OUTPUT_DIR = './'
else:
    OUTPUT_DIR = './nb001t-token-classifier/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Mounted at /content/drive
/content/drive/MyDrive/colab_notebooks/kaggle/nbme-score-clinical-patient-notes/notebooks


# CFG

In [4]:
# ====================================================
# CFG
# ====================================================

class CFG:
    pseudo=False
    wandb=True
    wandbgroup="nb001t-token-classifier"
    wandbname="case-num-all"
    cv_case_num=False
    competition='NBME'
    _wandb_kernel='riow1983'
    debug=False
    apex=True
    print_freq=100
    num_workers=8
    model="microsoft/deberta-v3-xsmall" #"microsoft/deberta-v3-large"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=15
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=16
    fc_dropout=0.2
    max_len=354
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0,1,2,3,4,]
    train=True
    num_subset=1 # -1 if not concatenting subsets of PL test data, 1 otherwise
    multi_pl=True # True if multiple PLs are created, False otherwise
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]
    CFG.wandbname = "debug-" + CFG.wandbname

In [5]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    if 'google.colab' in sys.modules:
        !pip install wandb
    import wandb

    try:
        if 'kaggle_web_client' in sys.modules:
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value_0 = user_secrets.get_secret("wandb_api")
        else:
            import json
            f = open("../../wandb.json", "r")
            json_data = json.load(f)
            secret_value_0 = json_data["wandb_api"]
        wandb.login(key=secret_value_0)
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(dir=OUTPUT_DIR,
                    project='NBME-Public', 
                    name=CFG.wandbname,
                    config=class2dict(CFG),
                    group=CFG.wandbgroup,
                    job_type="train",
                    anonymous=anony)
    print(f"wandb run id: {run.id}")

     |████████████████████████████████| 1.8 MB 13.7 MB/s 
     |████████████████████████████████| 144 kB 73.1 MB/s 
     |████████████████████████████████| 181 kB 61.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ec611c5caa4a949e92ee06d2ec15ca131ed46890754faf618c3b4e0eacc1330b
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: riow1983 (use `wandb login --relogin` to force relogin)


wandb run id: 1jsyqkkx


# Library

In [6]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold


os.system('pip uninstall -y torch torchvision torchaudio')
os.system('pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html')

import torch
print('torch.__version__:', torch.__version__)
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('python -m pip install --no-index --find-links=../input/nbme-pip-wheels transformers')
# os.system('pip install transformers')
os.system('pip install sentencepiece')

torch.__version__: 1.10.1+cu111


0

In [7]:
#%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:250

In [8]:
#!grep -e ^.*[tT]ransformer[a-zA-Z]+$

In [9]:
# !ls -l -a /usr/local/lib/python3.7/dist-packages/

In [10]:
# The following is necessary if you want to use the fast tokenizer for deberta v2 or v3
# This must be done before importing transformers
import shutil
from pathlib import Path

if 'kaggle_web_client' in sys.modules:
    transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")
elif "google.colab" in sys.modules:
    transformers_path = Path("/usr/local/lib/python3.7/dist-packages/transformers")
else:
    transformers_path = Path("")
    raise ValueError('Please specify your transformers path.')

input_dir = Path("../input/deberta-v2-3-fast-tokenizer")

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
    filepath = deberta_v2_path/filename
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

In [11]:
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.11.0
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


# Helper functions for scoring

In [12]:
# From https://www.kaggle.com/theoviel/evaluation-metric-folds-baseline

def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

In [13]:
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
    return results


def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
    return results


def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
    return predictions

# Utils

In [14]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = span_micro_f1(y_true, y_pred)
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# Data Loading

In [15]:
# ====================================================
# Data Loading
# ====================================================
if not CFG.pseudo:
    train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
    train['annotation'] = train['annotation'].apply(ast.literal_eval)
    train['location'] = train['location'].apply(ast.literal_eval)
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features = preprocess_features(features)
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')

if not CFG.pseudo:
    print(f"train.shape: {train.shape}")
    display(train.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

train.shape: (14300, 6)


,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724]
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693]
2,00016_002,0,16,2,[chest pressure],[203 217]
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]"
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258]


features.shape: (143, 3)


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


patient_notes.shape: (42146, 3)


,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [16]:
if not CFG.pseudo:
    train = train.merge(features, on=['feature_num', 'case_num'], how='left')
    train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
    display(train.head())

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,HPI: 17yo M presents with palpitations. Patien...


In [17]:
if not CFG.pseudo:
    # incorrect annotation
    train.loc[338, 'annotation'] = ast.literal_eval('[["father heart attack"]]')
    train.loc[338, 'location'] = ast.literal_eval('[["764 783"]]')

    train.loc[621, 'annotation'] = ast.literal_eval('[["for the last 2-3 months"]]')
    train.loc[621, 'location'] = ast.literal_eval('[["77 100"]]')

    train.loc[655, 'annotation'] = ast.literal_eval('[["no heat intolerance"], ["no cold intolerance"]]')
    train.loc[655, 'location'] = ast.literal_eval('[["285 292;301 312"], ["285 287;296 312"]]')

    train.loc[1262, 'annotation'] = ast.literal_eval('[["mother thyroid problem"]]')
    train.loc[1262, 'location'] = ast.literal_eval('[["551 557;565 580"]]')

    train.loc[1265, 'annotation'] = ast.literal_eval('[[\'felt like he was going to "pass out"\']]')
    train.loc[1265, 'location'] = ast.literal_eval('[["131 135;181 212"]]')

    train.loc[1396, 'annotation'] = ast.literal_eval('[["stool , with no blood"]]')
    train.loc[1396, 'location'] = ast.literal_eval('[["259 280"]]')

    train.loc[1591, 'annotation'] = ast.literal_eval('[["diarrhoe non blooody"]]')
    train.loc[1591, 'location'] = ast.literal_eval('[["176 184;201 212"]]')

    train.loc[1615, 'annotation'] = ast.literal_eval('[["diarrhea for last 2-3 days"]]')
    train.loc[1615, 'location'] = ast.literal_eval('[["249 257;271 288"]]')

    train.loc[1664, 'annotation'] = ast.literal_eval('[["no vaginal discharge"]]')
    train.loc[1664, 'location'] = ast.literal_eval('[["822 824;907 924"]]')

    train.loc[1714, 'annotation'] = ast.literal_eval('[["started about 8-10 hours ago"]]')
    train.loc[1714, 'location'] = ast.literal_eval('[["101 129"]]')

    train.loc[1929, 'annotation'] = ast.literal_eval('[["no blood in the stool"]]')
    train.loc[1929, 'location'] = ast.literal_eval('[["531 539;549 561"]]')

    train.loc[2134, 'annotation'] = ast.literal_eval('[["last sexually active 9 months ago"]]')
    train.loc[2134, 'location'] = ast.literal_eval('[["540 560;581 593"]]')

    train.loc[2191, 'annotation'] = ast.literal_eval('[["right lower quadrant pain"]]')
    train.loc[2191, 'location'] = ast.literal_eval('[["32 57"]]')

    train.loc[2553, 'annotation'] = ast.literal_eval('[["diarrhoea no blood"]]')
    train.loc[2553, 'location'] = ast.literal_eval('[["308 317;376 384"]]')

    train.loc[3124, 'annotation'] = ast.literal_eval('[["sweating"]]')
    train.loc[3124, 'location'] = ast.literal_eval('[["549 557"]]')

    train.loc[3858, 'annotation'] = ast.literal_eval('[["previously as regular"], ["previously eveyr 28-29 days"], ["previously lasting 5 days"], ["previously regular flow"]]')
    train.loc[3858, 'location'] = ast.literal_eval('[["102 123"], ["102 112;125 141"], ["102 112;143 157"], ["102 112;159 171"]]')

    train.loc[4373, 'annotation'] = ast.literal_eval('[["for 2 months"]]')
    train.loc[4373, 'location'] = ast.literal_eval('[["33 45"]]')

    train.loc[4763, 'annotation'] = ast.literal_eval('[["35 year old"]]')
    train.loc[4763, 'location'] = ast.literal_eval('[["5 16"]]')

    train.loc[4782, 'annotation'] = ast.literal_eval('[["darker brown stools"]]')
    train.loc[4782, 'location'] = ast.literal_eval('[["175 194"]]')

    train.loc[4908, 'annotation'] = ast.literal_eval('[["uncle with peptic ulcer"]]')
    train.loc[4908, 'location'] = ast.literal_eval('[["700 723"]]')

    train.loc[6016, 'annotation'] = ast.literal_eval('[["difficulty falling asleep"]]')
    train.loc[6016, 'location'] = ast.literal_eval('[["225 250"]]')

    train.loc[6192, 'annotation'] = ast.literal_eval('[["helps to take care of aging mother and in-laws"]]')
    train.loc[6192, 'location'] = ast.literal_eval('[["197 218;236 260"]]')

    train.loc[6380, 'annotation'] = ast.literal_eval('[["No hair changes"], ["No skin changes"], ["No GI changes"], ["No palpitations"], ["No excessive sweating"]]')
    train.loc[6380, 'location'] = ast.literal_eval('[["480 482;507 519"], ["480 482;499 503;512 519"], ["480 482;521 531"], ["480 482;533 545"], ["480 482;564 582"]]')

    train.loc[6562, 'annotation'] = ast.literal_eval('[["stressed due to taking care of her mother"], ["stressed due to taking care of husbands parents"]]')
    train.loc[6562, 'location'] = ast.literal_eval('[["290 320;327 337"], ["290 320;342 358"]]')

    train.loc[6862, 'annotation'] = ast.literal_eval('[["stressor taking care of many sick family members"]]')
    train.loc[6862, 'location'] = ast.literal_eval('[["288 296;324 363"]]')

    train.loc[7022, 'annotation'] = ast.literal_eval('[["heart started racing and felt numbness for the 1st time in her finger tips"]]')
    train.loc[7022, 'location'] = ast.literal_eval('[["108 182"]]')

    train.loc[7422, 'annotation'] = ast.literal_eval('[["first started 5 yrs"]]')
    train.loc[7422, 'location'] = ast.literal_eval('[["102 121"]]')

    train.loc[8876, 'annotation'] = ast.literal_eval('[["No shortness of breath"]]')
    train.loc[8876, 'location'] = ast.literal_eval('[["481 483;533 552"]]')

    train.loc[9027, 'annotation'] = ast.literal_eval('[["recent URI"], ["nasal stuffines, rhinorrhea, for 3-4 days"]]')
    train.loc[9027, 'location'] = ast.literal_eval('[["92 102"], ["123 164"]]')

    train.loc[9938, 'annotation'] = ast.literal_eval('[["irregularity with her cycles"], ["heavier bleeding"], ["changes her pad every couple hours"]]')
    train.loc[9938, 'location'] = ast.literal_eval('[["89 117"], ["122 138"], ["368 402"]]')

    train.loc[9973, 'annotation'] = ast.literal_eval('[["gaining 10-15 lbs"]]')
    train.loc[9973, 'location'] = ast.literal_eval('[["344 361"]]')

    train.loc[10513, 'annotation'] = ast.literal_eval('[["weight gain"], ["gain of 10-16lbs"]]')
    train.loc[10513, 'location'] = ast.literal_eval('[["600 611"], ["607 623"]]')

    train.loc[11551, 'annotation'] = ast.literal_eval('[["seeing her son knows are not real"]]')
    train.loc[11551, 'location'] = ast.literal_eval('[["386 400;443 461"]]')

    train.loc[11677, 'annotation'] = ast.literal_eval('[["saw him once in the kitchen after he died"]]')
    train.loc[11677, 'location'] = ast.literal_eval('[["160 201"]]')

    train.loc[12124, 'annotation'] = ast.literal_eval('[["tried Ambien but it didnt work"]]')
    train.loc[12124, 'location'] = ast.literal_eval('[["325 337;349 366"]]')

    train.loc[12279, 'annotation'] = ast.literal_eval('[["heard what she described as a party later than evening these things did not actually happen"]]')
    train.loc[12279, 'location'] = ast.literal_eval('[["405 459;488 524"]]')

    train.loc[12289, 'annotation'] = ast.literal_eval('[["experienced seeing her son at the kitchen table these things did not actually happen"]]')
    train.loc[12289, 'location'] = ast.literal_eval('[["353 400;488 524"]]')

    train.loc[13238, 'annotation'] = ast.literal_eval('[["SCRACHY THROAT"], ["RUNNY NOSE"]]')
    train.loc[13238, 'location'] = ast.literal_eval('[["293 307"], ["321 331"]]')

    train.loc[13297, 'annotation'] = ast.literal_eval('[["without improvement when taking tylenol"], ["without improvement when taking ibuprofen"]]')
    train.loc[13297, 'location'] = ast.literal_eval('[["182 221"], ["182 213;225 234"]]')

    train.loc[13299, 'annotation'] = ast.literal_eval('[["yesterday"], ["yesterday"]]')
    train.loc[13299, 'location'] = ast.literal_eval('[["79 88"], ["409 418"]]')

    train.loc[13845, 'annotation'] = ast.literal_eval('[["headache global"], ["headache throughout her head"]]')
    train.loc[13845, 'location'] = ast.literal_eval('[["86 94;230 236"], ["86 94;237 256"]]')

    train.loc[14083, 'annotation'] = ast.literal_eval('[["headache generalized in her head"]]')
    train.loc[14083, 'location'] = ast.literal_eval('[["56 64;156 179"]]')

In [18]:
# if CFG.pseudo:
#     print(train.shape)
#     # concatenate train w/ test(pseudo-labeld data)
#     if CFG.num_subset == -1:
#         pl = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}.csv")
#     else:
#         print('Concatenating subsets of test data...')
#         pl0 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub0.csv")
#         pl1 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub1.csv")
#         pl2 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub2.csv")
#         pl = pd.concat([pl0, pl1, pl2], axis=0, ignore_index=True)
#         pl_ = pl.drop_duplicates()
#         assert len(pl) == len(pl_), 'pl has duplicated rows!'
#         del pl0, pl1, pl2, pl_; gc.collect()

#     pl['annotation'] = pl['annotation'].apply(ast.literal_eval)
#     pl['location'] = pl['location'].apply(ast.literal_eval)
#     pl['id'] = pl['pn_num'].apply(lambda x: str(x).zfill(5))+'_'+pl['feature_num'].apply(lambda x: str(x).zfill(3))
#     pl = pl[train.columns]
#     #display(pl.head())
#     train['is_pl'] = 0
#     pl['is_pl'] = 1
#     train = pd.concat([train, pl], axis=0, ignore_index=True)
#     del pl
#     print(train.shape)

In [19]:
# # PL rows have [''] instead of []
# # must be changed to [] for 'annotation' and 'location'
# train['annotation'] = train['annotation'].apply(lambda x: [] if x==[''] else x)
# train['location'] = train['location'].apply(lambda x: [] if x==[''] else x)

In [20]:
# train['annotation_length'] = train['annotation'].apply(len)
# display(train['annotation_length'].value_counts())

## Merge patient_notes w/ features

In [21]:
print(patient_notes.shape)
patient_notes = patient_notes.merge(features, on=['case_num'], how='left')
print(patient_notes.shape)
display(patient_notes.head())

(42146, 3)
(626902, 5)


,pn_num,case_num,pn_history,feature_num,feature_text
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded


## ~~Remove pn_nums which are appeared in train from patient_notes~~

In [22]:
# print(patient_notes.shape)
# patient_notes = patient_notes[~patient_notes["pn_num"].isin(train["pn_num"].unique())].reset_index(drop=True)
# print(patient_notes.shape)

## Select one specific case_num

In [23]:
if CFG.wandbname.split("-")[-1] != "all":
    selected_case_num = int(CFG.wandbname.split("-")[-1])
    print(f"selected_case_num: {selected_case_num}")

    print(train.shape)
    train = train[train["case_num"]==selected_case_num].reset_index(drop=True)
    print(train.shape)

    print()

    print(patient_notes.shape)
    patient_notes = patient_notes[patient_notes["case_num"]==selected_case_num].reset_index(drop=True)
    print(patient_notes.shape)

    print()

    print(features.shape)
    features = features[features["case_num"]==selected_case_num].reset_index(drop=True)
    print(features.shape)

# CV split

In [24]:
# ====================================================
# CV split
# ====================================================
if not CFG.pseudo:
    Fold = GroupKFold(n_splits=CFG.n_fold)
    groups = train['pn_num'].values
    for n, (train_index, val_index) in enumerate(Fold.split(train, train['location'], groups)):
        train.loc[val_index, 'fold'] = int(n)
    train['fold'] = train['fold'].astype(int)
else:
    train = pd.read_pickle(OUTPUT_DIR+'train_folded.pkl')





if CFG.pseudo:
    print(train.shape)
    # concatenate train w/ test(pseudo-labeld data)
    if CFG.num_subset == -1:
        if CFG.multi_pl:
            res = []
            for i in range(CFG.n_fold):
                res.append(pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_f{i}.csv"))
            pl = pd.concat(res, axis=0, ignore_index=True)
            del res; gc.collect()

        else:
            pl = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}.csv")
    else:
        if CFG.multi_pl:
            res = []
            for i in range(CFG.n_fold):
                print('Concatenating subsets of test data...')
                pl0 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub0_f{i}.csv")
                pl1 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub1_f{i}.csv")
                pl2 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub2_f{i}.csv")
                pl = pd.concat([pl0, pl1, pl2], axis=0, ignore_index=True)
                pl_ = pl.drop_duplicates()
                assert len(pl) == len(pl_), 'pl has duplicated rows!'
                del pl0, pl1, pl2, pl_; gc.collect()
                res.append(pl)
            pl = pd.concat(res, axis=0, ignore_index=True)
            del res; gc.collect()

        else:
            print('Concatenating subsets of test data...')
            pl0 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub0.csv")
            pl1 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub1.csv")
            pl2 = pd.read_csv(f"./nb002i-token-classifier/test_{CFG.wandbname.split('-')[-1]}_sub2.csv")
            pl = pd.concat([pl0, pl1, pl2], axis=0, ignore_index=True)
            pl_ = pl.drop_duplicates()
            assert len(pl) == len(pl_), 'pl has duplicated rows!'
            del pl0, pl1, pl2, pl_; gc.collect()

    pl['annotation'] = pl['annotation'].apply(ast.literal_eval)
    pl['location'] = pl['location'].apply(ast.literal_eval)
    pl['id'] = pl['pn_num'].apply(lambda x: str(x).zfill(5))+'_'+pl['feature_num'].apply(lambda x: str(x).zfill(3))
    
    
    # PL rows have [''] instead of []
    # must be changed to [] for 'annotation' and 'location'
    pl['annotation'] = pl['annotation'].apply(lambda x: [] if x==[''] else x)
    pl['location'] = pl['location'].apply(lambda x: [] if x==[''] else x)

    if CFG.pseudo:
        pl['annotation_length'] = pl['annotation'].apply(len)
    # PL also has to have a column named "fold" when created by nb002i-token-classifier.ipynb
    # fold should be null if CFG.multi_pl is False
    # Otherwise, fold should be {10, 11, 12, 13, 14} if n_fold==5
    pl = pl[train.columns]
    #display(pl.head())
    train['is_pl'] = 0
    pl['is_pl'] = 1
    
    # Concatenate train and PL
    train = pd.concat([train, pl], axis=0, ignore_index=True)
    del pl
    print(train.shape)

if not CFG.pseudo:    
    train['annotation_length'] = train['annotation'].apply(len)

display(train['annotation_length'].value_counts())


#### Deprecated ####
# Deprecated as CV slit has been moved before trian-pl concatenation
# CV split does not applied to PL anymore

# if not CFG.multi_pl:
#     if CFG.pseudo:
#         # remove pseudo labels from val
#         train.loc[train['is_pl']==1, 'fold'] = -1
#### DeprecatedDeprecated ####

display(train.groupby('fold').size())
print(len(train))

1    8185
0    4399
2    1292
3     287
4      99
5      27
6       9
7       1
8       1
Name: annotation_length, dtype: int64

fold
0    2860
1    2860
2    2860
3    2860
4    2860
dtype: int64

14300


In [25]:
if CFG.debug:
    display(train.groupby('fold').size())
    if len(train) > 2000:
        train = train.sample(n=2000, random_state=0).reset_index(drop=True)
        display(train.groupby('fold').size())

In [26]:
if not CFG.pseudo:
    # Export 5fold trian data
    train.to_pickle(OUTPUT_DIR+'train_folded.pkl')

# tokenizer

In [27]:
# ====================================================
# tokenizer
# ====================================================

# tokenizer = AutoTokenizer.from_pretrained(CFG.model)
# tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
# CFG.tokenizer = tokenizer


from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast
tokenizer = DebertaV2TokenizerFast.from_pretrained('../input/deberta-v2-fast-tokenizer')
# tokenizer = DebertaV2TokenizerFast.from_pretrained('../input/deberta-v2-3-fast-tokenizer')
CFG.tokenizer = tokenizer

# Dataset

In [28]:
# ====================================================
# Define max_len
# ====================================================
for text_col in ['pn_history']:
    pn_history_lengths = []
    tk0 = tqdm(patient_notes[text_col].fillna("").values, total=len(patient_notes))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False, max_length=CFG.max_len, truncation=True)['input_ids'])
        pn_history_lengths.append(length)
    LOGGER.info(f'{text_col} max(lengths): {max(pn_history_lengths)}')

for text_col in ['feature_text']:
    features_lengths = []
    tk0 = tqdm(features[text_col].fillna("").values, total=len(features))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False, max_length=CFG.max_len, truncation=True)['input_ids'])
        features_lengths.append(length)
    LOGGER.info(f'{text_col} max(lengths): {max(features_lengths)}')

CFG.max_len = max(pn_history_lengths) + max(features_lengths) + 3 # cls & sep & sep
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/626902 [00:00<?, ?it/s]

pn_history max(lengths): 323


  0%|          | 0/143 [00:00<?, ?it/s]

feature_text max(lengths): 28
max_len: 354


In [29]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text, feature_text):
    inputs = cfg.tokenizer(text, feature_text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


def create_label(cfg, text, annotation_length, location_list):
    encoded = cfg.tokenizer(text,
                            add_special_tokens=True,
                            max_length=CFG.max_len,
                            padding="max_length",
                            return_offsets_mapping=True)
    offset_mapping = encoded['offset_mapping']
    ignore_idxes = np.where(np.array(encoded.sequence_ids()) != 0)[0]
    label = np.zeros(len(offset_mapping))
    label[ignore_idxes] = -1
    if annotation_length != 0:
        for location in location_list:
            for loc in [s.split() for s in location.split(';')]:
                start_idx = -1
                end_idx = -1
                start, end = int(loc[0]), int(loc[1])
                for idx in range(len(offset_mapping)):
                    if (start_idx == -1) & (start < offset_mapping[idx][0]):
                        start_idx = idx - 1
                    if (end_idx == -1) & (end <= offset_mapping[idx][1]):
                        end_idx = idx + 1
                if start_idx == -1:
                    start_idx = end_idx
                if (start_idx != -1) & (end_idx != -1):
                    label[start_idx:end_idx] = 1
    return torch.tensor(label, dtype=torch.float)


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values
        self.annotation_lengths = df['annotation_length'].values
        self.locations = df['location'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item])
        label = create_label(self.cfg, 
                             self.pn_historys[item], 
                             self.annotation_lengths[item], 
                             self.locations[item])
        return inputs, label

# Model

In [30]:
# ====================================================
# Model
# ====================================================
# class CustomModel(nn.Module):
#     def __init__(self, cfg, config_path=None, pretrained=False):
#         super().__init__()
#         self.cfg = cfg
#         if config_path is None:
#             self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
#         else:
#             self.config = torch.load(config_path)
#         if pretrained:
#             self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
#         else:
#             self.model = AutoModel(self.config)
#         self.fc_dropout = nn.Dropout(cfg.fc_dropout)
#         self.fc = nn.Linear(self.config.hidden_size, 1)
#         self._init_weights(self.fc)
        
#     def _init_weights(self, module):
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.padding_idx is not None:
#                 module.weight.data[module.padding_idx].zero_()
#         elif isinstance(module, nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)
        
#     def feature(self, inputs):
#         outputs = self.model(**inputs)
#         last_hidden_states = outputs[0]
#         return last_hidden_states

#     def forward(self, inputs):
#         feature = self.feature(inputs)
#         output = self.fc(self.fc_dropout(feature))
#         return output



class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)

        if 'deberta-v3-xxlarge' in cfg.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:24].requires_grad_(False) # 冻结24/48
        if 'deberta-v3-xlarge' in cfg.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:12].requires_grad_(False) # 冻结12/24
        
        self.fc_dropout_0 = nn.Dropout(0.1)
        self.fc_dropout_1 = nn.Dropout(0.2)
        self.fc_dropout_2 = nn.Dropout(0.3)
        self.fc_dropout_3 = nn.Dropout(0.4)
        self.fc_dropout_4 = nn.Dropout(0.5)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output_0 = self.fc(self.fc_dropout_0(feature))
        output_1 = self.fc(self.fc_dropout_1(feature))
        output_2 = self.fc(self.fc_dropout_2(feature))
        output_3 = self.fc(self.fc_dropout_3(feature))
        output_4 = self.fc(self.fc_dropout_4(feature))
        output = (output_0 + output_1 + output_2 + output_3 + output_4) / 5
        return output

# Helper functions

In [31]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions


def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [32]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    if CFG.pseudo & CFG.multi_pl:
        pl_cond = (folds['is_pl']==1)&(folds['fold']==fold+10)
        npl_cond = folds['is_pl']==0
        folds_pl = folds.loc[pl_cond, :].copy()
        folds_npl = folds.loc[npl_cond, :].copy()
        folds = pd.concat([folds_pl, folds_npl], axis=0, ignore_index=True)
        del folds_pl, folds_npl; gc.collect()
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    
    
    valid_texts = valid_folds['pn_history'].values
    valid_labels = create_labels_for_scoring(valid_folds)
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.BCEWithLogitsLoss(reduction="none")
    
    best_score = 0.

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        predictions = predictions.reshape((len(valid_folds), CFG.max_len))
        
        # scoring
        char_probs = get_char_probs(valid_texts, predictions, CFG.tokenizer)
        results = get_results(char_probs, th=0.5)
        preds = get_predictions(results)
        score = get_score(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score < score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            if CFG.pseudo:
                torch.save({'model': model.state_dict(),
                            'predictions': predictions},
                            OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best_pl.pth")
            else:
                torch.save({'model': model.state_dict(),
                            'predictions': predictions},
                            OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
    
    if CFG.pseudo:
        predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best_pl.pth", 
                                map_location=torch.device('cpu'))['predictions']
    else:
        predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                                map_location=torch.device('cpu'))['predictions']
    valid_folds[[i for i in range(CFG.max_len)]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [33]:
if __name__ == '__main__':
    
    def get_result(oof_df, cv_score=False, case_num=None):
        if case_num is not None:
            oof_df = oof_df[oof_df["case_num"]==case_num].reset_index(drop=True)
        labels = create_labels_for_scoring(oof_df)
        predictions = oof_df[[i for i in range(CFG.max_len)]].values
        char_probs = get_char_probs(oof_df['pn_history'].values, predictions, CFG.tokenizer)
        results = get_results(char_probs, th=0.5)
        preds = get_predictions(results)
        score = get_score(labels, preds)
        if case_num is not None:
            LOGGER.info(f'Score of case_num {case_num}: {score:<.4f}')
            if cv_score:
                wandb.log({f'CV score of case_num {case_num}': score})
        else:
            LOGGER.info(f'Score: {score:<.4f}')
            if cv_score:
                wandb.log({f'CV score': score})
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                _oof_df.to_pickle(OUTPUT_DIR+f'_oof_df_f{fold}.pkl')
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        if CFG.cv_case_num:
            for i in range(10):
                get_result(oof_df, cv_score=True, case_num=i)
        else:
            get_result(oof_df, cv_score=True)
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        
    if CFG.wandb:
        wandb.finish()


    # Push to LINE
    import requests
    def send_line_notification(message):
        import json
        f = open("../../line.json", "r")
        json_data = json.load(f)
        line_token = json_data["kagglePush"]
        endpoint = 'https://notify-api.line.me/api/notify'
        message = "\n{}".format(message)
        payload = {'message': message}
        headers = {'Authorization': 'Bearer {}'.format(line_token)}
        requests.post(endpoint, data=payload, headers=headers)

    if CFG.wandb:
        send_line_notification(f"Training of {CFG.wandbgroup} has been done. See {run.url}")
    else:
        send_line_notification(f"Training of {CFG.wandbgroup} has been done.")

========== fold: 0 training ==========


Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-xsmall were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'deberta.embeddings.word_embeddings._weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequ

Epoch: [1][0/715] Elapsed 0m 0s (remain 9m 5s) Loss: 0.5963(0.5963) Grad: inf  LR: 0.00002000  
Epoch: [1][100/715] Elapsed 0m 15s (remain 1m 37s) Loss: 0.0305(0.1328) Grad: 1349.1963  LR: 0.00002000  
Epoch: [1][200/715] Elapsed 0m 31s (remain 1m 19s) Loss: 0.0347(0.0882) Grad: 4241.9541  LR: 0.00001998  
Epoch: [1][300/715] Elapsed 0m 46s (remain 1m 3s) Loss: 0.0246(0.0707) Grad: 2484.1748  LR: 0.00001996  
Epoch: [1][400/715] Elapsed 1m 1s (remain 0m 48s) Loss: 0.0332(0.0613) Grad: 1977.3671  LR: 0.00001993  
Epoch: [1][500/715] Elapsed 1m 16s (remain 0m 32s) Loss: 0.0287(0.0548) Grad: 2068.2883  LR: 0.00001989  
Epoch: [1][600/715] Elapsed 1m 31s (remain 0m 17s) Loss: 0.0216(0.0501) Grad: 3032.7820  LR: 0.00001985  
Epoch: [1][700/715] Elapsed 1m 46s (remain 0m 2s) Loss: 0.0313(0.0465) Grad: 4891.5918  LR: 0.00001979  
Epoch: [1][714/715] Elapsed 1m 48s (remain 0m 0s) Loss: 0.0295(0.0460) Grad: 2224.2332  LR: 0.00001978  
EVAL: [0/179] Elapsed 0m 0s (remain 1m 35s) Loss: 0.0119(0.0

...

Epoch: [14][0/715] Elapsed 0m 0s (remain 8m 1s) Loss: 0.0060(0.0060) Grad: 10937.6699  LR: 0.00000086  
Epoch: [14][100/715] Elapsed 0m 16s (remain 1m 37s) Loss: 0.0089(0.0092) Grad: 12483.7080  LR: 0.00000075  
Epoch: [14][200/715] Elapsed 0m 31s (remain 1m 20s) Loss: 0.0053(0.0087) Grad: 13465.9385  LR: 0.00000064  
Epoch: [14][300/715] Elapsed 0m 46s (remain 1m 4s) Loss: 0.0206(0.0092) Grad: 38497.2305  LR: 0.00000054  
Epoch: [14][400/715] Elapsed 1m 1s (remain 0m 48s) Loss: 0.0092(0.0092) Grad: 14406.5137  LR: 0.00000045  
Epoch: [14][500/715] Elapsed 1m 17s (remain 0m 32s) Loss: 0.0037(0.0090) Grad: 10685.8896  LR: 0.00000037  
Epoch: [14][600/715] Elapsed 1m 32s (remain 0m 17s) Loss: 0.0091(0.0091) Grad: 12812.0684  LR: 0.00000029  
Epoch: [14][700/715] Elapsed 1m 47s (remain 0m 2s) Loss: 0.0091(0.0091) Grad: 18600.8516  LR: 0.00000023  
Epoch: [14][714/715] Elapsed 1m 49s (remain 0m 0s) Loss: 0.0062(0.0091) Grad: 12799.0996  LR: 0.00000022  
EVAL: [0/179] Elapsed 0m 0s (remain 

Epoch 14 - avg_train_loss: 0.0091  avg_val_loss: 0.0156  time: 126s
Epoch 14 - Score: 0.8459
Epoch 14 - Save Best Score: 0.8459 Model


Epoch: [15][0/715] Elapsed 0m 0s (remain 8m 4s) Loss: 0.0034(0.0034) Grad: 6750.9429  LR: 0.00000022  
Epoch: [15][100/715] Elapsed 0m 15s (remain 1m 36s) Loss: 0.0115(0.0098) Grad: 26815.3086  LR: 0.00000016  
Epoch: [15][200/715] Elapsed 0m 30s (remain 1m 19s) Loss: 0.0037(0.0099) Grad: 4545.8965  LR: 0.00000011  
Epoch: [15][300/715] Elapsed 0m 45s (remain 1m 3s) Loss: 0.0064(0.0095) Grad: 12050.8057  LR: 0.00000007  
Epoch: [15][400/715] Elapsed 1m 0s (remain 0m 47s) Loss: 0.0034(0.0095) Grad: 9554.6123  LR: 0.00000004  
Epoch: [15][500/715] Elapsed 1m 16s (remain 0m 32s) Loss: 0.0021(0.0092) Grad: 8643.0352  LR: 0.00000002  
Epoch: [15][600/715] Elapsed 1m 30s (remain 0m 17s) Loss: 0.0054(0.0091) Grad: 7222.2305  LR: 0.00000001  
Epoch: [15][700/715] Elapsed 1m 45s (remain 0m 2s) Loss: 0.0075(0.0091) Grad: 17407.4160  LR: 0.00000000  
Epoch: [15][714/715] Elapsed 1m 48s (remain 0m 0s) Loss: 0.0024(0.0091) Grad: 8262.6523  LR: 0.00000000  
EVAL: [0/179] Elapsed 0m 0s (remain 1m 37s

Epoch 15 - avg_train_loss: 0.0091  avg_val_loss: 0.0156  time: 124s
Epoch 15 - Score: 0.8451
========== fold: 4 result ==========
Score: 0.8459
========== CV ==========
Score: 0.8483


CV score,▁
[fold0] avg_train_loss,█▃▃▃▂▂▂▂▁▁▁▁▁▁▁
[fold0] avg_val_loss,█▆▄▃▃▂▁▁▁▁▁▁▁▁▁
[fold0] epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
[fold0] loss,█▄▄▄▃▃▃▃▃▂▅▄▃▂▆▅▃▃▂▁▄▁▂▂▃▂▄▁▁▄▅▂▂▃▂▂▂▂▁▁
[fold0] lr,███████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
[fold0] score,▁▃▄▅▆▇▇▇███████
[fold1] avg_train_loss,█▃▃▂▂▂▂▁▁▁▁▁▁▁▁
[fold1] avg_val_loss,█▆▄▃▂▂▁▁▁▁▁▁▂▂▂
[fold1] epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
[fold1] loss,█▃▂▂▃▃▃▂▂▂▂▃▂▂▁▂▂▁▂▂▂▂▂▁▂▂▁▂▁▁▂▂▁▁▂▂▁▁▂▁
